Using larger models, comes with the expense of larger memory expense (primarily during gradient calculations)

To deal with the out-of-memory issue

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
try: import fastkaggle
except ModuleNotFoundError:
    !pip install -Uq fastkaggle

from fastkaggle import *

In [3]:
from fastai.imports import *
np.set_printoptions(linewidth=130)
import os
from pathlib import Path
path = Path('/kaggle/input/paddy-doctor/paddy-disease-classification')
path

Path('/kaggle/input/paddy-doctor/paddy-disease-classification')

In [4]:
from fastai.vision.all import *
set_seed(42)

path.ls()

(#6) [Path('/kaggle/input/paddy-doctor/paddy-disease-classification/sample_submission.csv'),Path('/kaggle/input/paddy-doctor/paddy-disease-classification/train_images'),Path('/kaggle/input/paddy-doctor/paddy-disease-classification/.jovianrc'),Path('/kaggle/input/paddy-doctor/paddy-disease-classification/.ipynb_checkpoints'),Path('/kaggle/input/paddy-doctor/paddy-disease-classification/train.csv'),Path('/kaggle/input/paddy-doctor/paddy-disease-classification/test_images')]

In [5]:
tst_files = get_image_files(path/'test_images').sorted()

In [6]:
df = pd.read_csv(path/'train.csv')
df.label.value_counts()

label
normal                      1764
blast                       1738
hispa                       1594
dead_heart                  1442
tungro                      1088
brown_spot                   965
downy_mildew                 620
bacterial_leaf_blight        479
bacterial_leaf_streak        380
bacterial_panicle_blight     337
Name: count, dtype: int64

In [7]:
trn_path = path/'train_images'/'bacterial_panicle_blight'

In [8]:
import timm

def train(arch, size, item=Resize(480, method='squish'), accum=1, finetune=True, epochs=12):
    dls = ImageDataLoaders.from_folder(
        trn_path, valid_pct=0.2, item_tfms=item,
        batch_tfms=aug_transforms(size=size, min_scale=0.75),
        bs=64//accum
    )

    cbs = GradientAccumulation(64) if accum else []

    # Build the model directly in timm with the correct number of classes
    model = timm.create_model(arch, pretrained=True, num_classes=dls.c)

    # Standard fastai Learner on that model (no fastai auto-head)
    learn = Learner(
        dls, model, loss_func=CrossEntropyLossFlat(), metrics=error_rate, cbs=cbs
    ).to_fp16()

    if finetune:
        # mimic fine_tune: train head a bit, then unfreeze
        learn.freeze()
        learn.fit_one_cycle(1, 1e-3)
        learn.unfreeze()
        learn.fit_one_cycle(epochs-1 if epochs > 1 else 1, 1e-4)
        return learn.tta(dl=dls.test_dl(tst_files))
    else:
        learn.fit_one_cycle(epochs, 1e-3)

In [9]:
train('convnext_small_in22k', 128, epochs=1, accum=1, finetune=False)

/usr/local/lib/python3.11/dist-packages/timm/models/_factory.py:126: UserWarning: Mapping deprecated model name convnext_small_in22k to current convnext_small.fb_in22k.
  model = create_fn(


model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.11/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:10


In [10]:
import gc
def report_gpu():
    print(torch.cuda.list_gpu_processes())
    gc.collect()
    torch.cuda.empty_cache()

In [11]:
report_gpu()

GPU:0
process       3648 uses     3364.000 MB GPU memory


In [12]:
train('convnext_small_in22k', 128, epochs=1, accum=2, finetune=False)
report_gpu()

epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:11


GPU:0
process       3648 uses     2322.000 MB GPU memory


In [13]:
train('convnext_small_in22k', 128, epochs=1, accum=4, finetune=False)
report_gpu()

epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:19


GPU:0
process       3648 uses     1804.000 MB GPU memory


In [14]:
report_gpu()

GPU:0
process       3648 uses      388.000 MB GPU memory


In [15]:
train('convnext_large_in22k', 224, epochs=1, accum=2, finetune=False)
report_gpu()

/usr/local/lib/python3.11/dist-packages/timm/models/_factory.py:126: UserWarning: Mapping deprecated model name convnext_large_in22k to current convnext_large.fb_in22k.
  model = create_fn(


model.safetensors:   0%|          | 0.00/919M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:32


GPU:0
process       3648 uses    10228.000 MB GPU memory


In [16]:
report_gpu()

GPU:0
process       3648 uses      388.000 MB GPU memory


In [17]:
train('convnext_large_in22k', (320,240), epochs=1, accum=2, finetune=False)
report_gpu()

epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:39


GPU:0
process       3648 uses    13538.000 MB GPU memory


In [18]:
report_gpu()

GPU:0
process       3648 uses      390.000 MB GPU memory


In [19]:
train('vit_large_patch16_224', 224, epochs=1, accum=2, finetune=False)
report_gpu()

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:17


GPU:0
process       3648 uses    11762.000 MB GPU memory


In [20]:
report_gpu()

GPU:0
process       3648 uses      388.000 MB GPU memory


In [21]:
train('swinv2_large_window12_192_22k', 192, epochs=1, accum=2, finetune=False)
report_gpu()

/usr/local/lib/python3.11/dist-packages/timm/models/_factory.py:126: UserWarning: Mapping deprecated model name swinv2_large_window12_192_22k to current swinv2_large_window12_192.ms_in22k.
  model = create_fn(


model.safetensors:   0%|          | 0.00/917M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:11


GPU:0
process       3648 uses    12714.000 MB GPU memory


In [22]:
train('swin_large_patch4_window7_224', 224, epochs=1, accum=2, finetune=False)
report_gpu()

model.safetensors:   0%|          | 0.00/788M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,00:12


GPU:0
process       3648 uses    11122.000 MB GPU memory


In [23]:
import timm
import torch
from fastai.vision.all import *

# Enable cudnn auto-tuner
torch.backends.cudnn.benchmark = True

def train(arch, size, item=Resize(480, method='squish'), accum=1, finetune=True, epochs=12, bs=64):
    # DataLoaders
    dls = ImageDataLoaders.from_folder(
        trn_path,
        valid_pct=0.2,
        item_tfms=item,
        batch_tfms=aug_transforms(size=size, min_scale=0.75),
        bs=bs//accum,
        num_workers=4,  # increase if Kaggle allows
        pin_memory=True
    )

    # Gradient accumulation callback
    cbs = [GradientAccumulation(accum)] if accum > 1 else []

    # Create timm model
    model = timm.create_model(arch, pretrained=True, num_classes=dls.c)

    # Learner
    learn = Learner(
        dls, model,
        loss_func=CrossEntropyLossFlat(),
        metrics=error_rate,
        cbs=cbs
    ).to_fp16()

    if finetune:
        # Mimic fastai fine_tune
        learn.freeze()
        learn.fit_one_cycle(1, 1e-3)
        learn.unfreeze()
        learn.fit_one_cycle(max(epochs-1, 1), 1e-4)
        return learn.tta(dl=dls.test_dl(tst_files))
    else:
        learn.fit_one_cycle(epochs, 1e-3)
        return learn  # return for ensemble use


In [24]:
res = 640,480

In [25]:
report_gpu()

GPU:0
process       3648 uses      388.000 MB GPU memory


In [26]:
models = {
    'convnext_large_in22k': {
        (Resize(res), (320,224)),
    }, 'vit_large_patch16_224': {
        (Resize(480, method='squish'), 224),
        (Resize(res), 224),
    }, 'swinv2_large_window12_192_22k': {
        (Resize(480, method='squish'), 192),
        (Resize(res), 192),
    }, 'swin_large_patch4_window7_224': {
        (Resize(res), 224),
    }
}

In [27]:
trn_path = path/'train_images'

In [28]:
tta_res = []

for arch, details in models.items():
    for item, size in details:
        print(f'--- {arch} | size {size} | item {item.name}')
        learn = train(arch, size, item=item, accum=2, epochs=1)
        tta_res.append(learn)
        gc.collect()
        torch.cuda.empty_cache()

--- convnext_large_in22k | size (320, 224) | item Resize -- {'size': (480, 640), 'method': 'crop', 'pad_mode': 'reflection', 'resamples': (<Resampling.BILINEAR: 2>, <Resampling.NEAREST: 0>), 'p': 1.0}


epoch,train_loss,valid_loss,error_rate,time
0,0.472151,0.316295,0.098510,11:42


epoch,train_loss,valid_loss,error_rate,time
0,0.187026,0.164280,0.051418,11:28


--- vit_large_patch16_224 | size 224 | item Resize -- {'size': (480, 640), 'method': 'crop', 'pad_mode': 'reflection', 'resamples': (<Resampling.BILINEAR: 2>, <Resampling.NEAREST: 0>), 'p': 1.0}


epoch,train_loss,valid_loss,error_rate,time
0,2.123993,2.053512,0.728015,08:50


...

epoch,train_loss,valid_loss,error_rate,time
0,1.869004,1.718959,0.621336,05:42


epoch,train_loss,valid_loss,error_rate,time
0,1.112241,0.954458,0.326766,05:42


--- swin_large_patch4_window7_224 | size 224 | item Resize -- {'size': (480, 640), 'method': 'crop', 'pad_mode': 'reflection', 'resamples': (<Resampling.BILINEAR: 2>, <Resampling.NEAREST: 0>), 'p': 1.0}


epoch,train_loss,valid_loss,error_rate,time
0,2.154639,2.154676,0.824123,05:55


epoch,train_loss,valid_loss,error_rate,time
0,2.073656,1.993422,0.727535,05:55


In [29]:
save_pickle('tta_res.pkl', tta_res)

In [30]:
tta_prs = first(zip(*tta_res))

In [31]:
tta_prs += tta_prs[1:3]

In [32]:
avg_pr = torch.stack(tta_prs).mean(0)
avg_pr.shape

torch.Size([3469, 10])

In [33]:
dls = ImageDataLoaders.from_folder(trn_path, valid_pct=0.2, item_tfms=Resize(480, method='squish'),
    batch_tfms=aug_transforms(size=224, min_scale=0.75))

In [34]:
idxs = avg_pr.argmax(dim=1)
vocab = np.array(dls.vocab)
ss = pd.read_csv(path/'sample_submission.csv')
ss['label'] = vocab[idxs]
ss.to_csv('subm.csv', index=False)

In [35]:
!head subm.csv

image_id,label
200001.jpg,hispa
200002.jpg,normal
200003.jpg,normal
200004.jpg,blast
200005.jpg,blast
200006.jpg,brown_spot
200007.jpg,dead_heart
200008.jpg,hispa
200009.jpg,hispa
